# Preenchimento híbrido
Mostra o comportamento da rede utilizando os dois tipos de preenchimento. Essa simulação registra informações das ``requests``:

    - ID da requisição 
    - Alice
    - Bob
    - Fidelidade Mínima
    - Número de EPRs
    - Início (Quando começou a ser tratada)
    - Término (Quando foi atendida)
    - Novo Registro (Se foi necessário criar uma regra para atender, ou se já foi registrado)

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request, clear_file
from quantumnet.objects import time

Informações para a simulação

In [2]:
# Caminho do arquivo de saída
path = "resultados/hibrido/sync.csv"
# Limpa o arquivo de saída
clear_file(path)
# Sobre a rede
rede = Network()
rede.n_initial_eprs = 10
rede.n_initial_qubits = 10
row, col = 3, 4
rede.set_ready_topology("Grade", (row, col))
# Sobre o controlador
controlador = Controller(rede)
controlador.default_ttl = 100
# Sobre as requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts), fmin_range, neprs_range) for i in range(num_requests))

Preenchimento proativo

In [ ]:
hosts = rede.hosts
for alice in hosts:
    print(f"Adicionando regras para {alice}")
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.5, 0.6), 5)
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.8, 0.9), 5)
    # Mostra as tabelas
    rede.get_host(alice).draw_flow_table()

Funções responsáveis pela passagem de tempo

In [4]:
def update_time(time_for_update):
    """
    Atualiza o tempo e as regras

    Args:
        time_for_update (int): Tempo para incrementar o time-slot.
    """
    # Atualiza o tempo
    for t in range(time_for_update):
        time.increment()
        # Atualiza os recursos de 10 em 10
        if time.get_current_time() % 10 == 0:
            rede.refresh_resources(num_qubits=10, num_eprs=10)
            print(f"[Time {time.get_current_time()}] Recursos atualizados")

Preenchimento reativo, baseado nas requests.

    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.

In [ ]:
for request in requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            update_time(3)
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time() 
            # Registra no CSV como um novo registro
            register_request(request, "Novo Registro", path)
        else:  # Caso já exista a regra
            request.starttime = time.get_current_time()
            update_time(1)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            # Registra no CSV como já registrado
            register_request(request, "Já Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

Registro de recursos

In [ ]:
rede.registry_of_resources

Tabelas ao final

In [ ]:
for alice in hosts:
    print(f"Regras de {alice}")
    rede.get_host(alice).draw_flow_table()